In [35]:
import os
import csv
import numpy as np
import pandas as pd
import zipfile
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
#unzip and read turns files into pandas df
zf1 = zipfile.ZipFile('./turns_part1.zip') 
turns1 = pd.read_csv(zf1.open('turns_part1.csv'))
zf2 = zipfile.ZipFile('./turns_part2.zip') 
turns2 = pd.read_csv(zf2.open('turns_part2.csv'))
zf3 = zipfile.ZipFile('./turns_part3.zip') 
turns3 = pd.read_csv(zf3.open('turns_part3.csv'))
zf4 = zipfile.ZipFile('./turns_part4.zip') 
turns4 = pd.read_csv(zf4.open('turns_part4.csv'))

In [4]:
###initial test using count vectorizer bag of words using not_a_justice and scotus_justice

#use original scdb winning party as verdict
#verdict value 0 = no favorable disposition for petitioning part apparent
#verdict value 1 = petitioning party received a favorable disposition
verdict = []
verdict_csv = csv.reader(open('../SCDB_2017_01_caseCentered_Citation.csv'))
for row in verdict_csv:
    docket_number = re.sub('-', '_', row[13])
    docket_number = re.sub(' ORIG', '_orig', docket_number)
    case_id = row[10]+'_'+docket_number
    verdict.append([case_id, row[36]])
    
verdict_header = verdict.pop(0)
verdict = pd.DataFrame(verdict, columns = verdict_header)
    
    
#concate the turns files
turns_combined = pd.concat([turns1, turns2, turns3, turns4])

#remove _t01 and _t02 from transcript_id in turns_combined
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t01','')
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t02','')

#pivot turns files by transcript_id and speaker_role for texts
speaker_role_text_pivot = turns_combined.pivot_table(index = 'transcript_id', 
                                                     columns = 'speaker_role', 
                                                     values = 'text',
                                                     aggfunc=lambda x: ' '.join(x))

#reset index
speaker_role_text_pivot = speaker_role_text_pivot.reset_index()

#join verdict into df
train_test_df = speaker_role_text_pivot.join(verdict.set_index('term_docket')['partyWinning'], on='transcript_id')

#remove NAs and blanks (these give errors when vectorizing)
train_test_df = train_test_df.dropna()

#create train and test split
x = train_test_df[['not_a_justice', 'scotus_justice']]
y = train_test_df.partyWinning
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

#count vectorizer
count_vect_1 = CountVectorizer()
count_vect_2 = CountVectorizer()
x_train_cv_not_a_justice = count_vect_1.fit(x_train['not_a_justice'])
x_train_cv_scotus_justice = count_vect_2.fit(x_train['scotus_justice'])
x_test_cv_not_a_justice = count_vect_1.fit(x_test['not_a_justice'])
x_test_cv_scotus_justice = count_vect_2.fit(x_test['scotus_justice'])

x_train_cv_not_a_justice = pd.DataFrame(x_train_cv_not_a_justice.transform(x_train['not_a_justice']).todense(),
                                        columns = x_train_cv_not_a_justice.get_feature_names())
x_train_cv_scotus_justice = pd.DataFrame(x_train_cv_scotus_justice.transform(x_train['scotus_justice']).todense(),
                                        columns = x_train_cv_scotus_justice.get_feature_names())
x_test_cv_not_a_justice = pd.DataFrame(x_test_cv_not_a_justice.transform(x_test['not_a_justice']).todense(),
                                        columns = x_test_cv_not_a_justice.get_feature_names())
x_test_cv_scotus_justice = pd.DataFrame(x_test_cv_scotus_justice.transform(x_test['scotus_justice']).todense(),
                                        columns = x_test_cv_scotus_justice.get_feature_names())

#concatenate the not_a_justice bow and scotus_justice bow
x_train = pd.concat([x_train_cv_not_a_justice, x_train_cv_scotus_justice], axis = 1)
x_test = pd.concat([x_test_cv_not_a_justice, x_test_cv_scotus_justice], axis = 1)

#random forest
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(x_train, y_train)
random_forest_prediction = forest.predict(x_test)
rf_accuracy = np.mean(random_forest_prediction == y_test)
print 'Accuracy of Bag-of-Words Random Forest Model:', round(rf_accuracy,4)

# Multinomial Naive Baynes
nb = MultinomialNB(alpha = 0.1).fit(x_train, y_train)
naive_baynes_prediction = nb.predict(x_test)
nb_accuracy = np.mean(naive_baynes_prediction == y_test)
print 'Accuracy of Bag-of-Words Multinomial Naive Baynes Model:', round(nb_accuracy,4)

# logistic regression
lr_model = LogisticRegression(C = 1, penalty = "l2")
lr_model_train = lr_model.fit(x_train, y_train)
lr_prediction = lr_model.predict(x_test)
lr_accuracy = np.mean(lr_prediction == y_test)
print 'Accuracy of Bag-of-Words Logistic Regression Model:', round(lr_accuracy,4)

print 'Actual Count of y_test Verdicts:\n', y_test.value_counts()

print 'LR predictions:\n', np.unique(lr_prediction, return_counts=True)

Accuracy of Bag-of-Words Random Forest Model: 0.6124
Accuracy of Bag-of-Words Multinomial Naive Baynes Model: 0.5511
Accuracy of Bag-of-Words Logistic Regression Model: 0.5647
Actual Count of y_test Verdicts:
1    1082
0     679
        1
Name: partyWinning, dtype: int64
LR predictions:
(array(['0', '1'], dtype=object), array([ 551, 1211]))


In [14]:
###tfidf vectorizer bag of words using scotus_justice, petitioner, respondent

#use original scdb winning party as verdict
#verdict value 0 = no favorable disposition for petitioning part apparent
#verdict value 1 = petitioning party received a favorable disposition
verdict = []
verdict_csv = csv.reader(open('../SCDB_2017_01_caseCentered_Citation.csv'))
for row in verdict_csv:
    docket_number = re.sub('-', '_', row[13])
    docket_number = re.sub(' ORIG', '_orig', docket_number)
    case_id = row[10]+'_'+docket_number
    verdict.append([case_id, row[36]])
    
verdict_header = verdict.pop(0)
verdict = pd.DataFrame(verdict, columns = verdict_header)
    
    
#concate the turns files
turns_combined = pd.concat([turns1, turns2, turns3, turns4])

#remove _t01 and _t02 from transcript_id in turns_combined
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t01','')
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t02','')

#get advocate sides
advocates = pd.read_json('./advocate_dict.json')

advocate_turns = []

for index, row in turns_combined.iterrows():
    if row['speaker_role'] == 'scotus_justice':
        advocate_turns.append('scotus_justice')
    else:
        speaker = row['speaker']
        transcript_id = row['transcript_id']
        try:
            lawyer_side = advocates.ix[speaker][transcript_id]
            advocate_turns.append(lawyer_side)
        except:
            advocate_turns.append('None')

#insert advocate side to the turns_combined dataframe
turns_combined['lawyer_side'] = advocate_turns

#pivot turns files by transcript_id and speaker_role for texts
speaker_role_text_pivot = turns_combined.pivot_table(index = 'transcript_id', 
                                                     columns = 'lawyer_side', 
                                                     values = 'text',
                                                     aggfunc=lambda x: ' '.join(x))
#reset index
speaker_role_text_pivot = speaker_role_text_pivot.reset_index()

#drop columns with no lawyer side tags
speaker_role_text_pivot = speaker_role_text_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#count of number of times speaker spoke
counts_pivot = pd.pivot_table(turns_combined[['transcript_id', 'lawyer_side']],
                              index = 'transcript_id',
                              columns = 'lawyer_side',
                              aggfunc=len,
                              fill_value=0)

#reset index
counts_pivot = counts_pivot.reset_index()

#drop columns with no lawyer side tags
counts_pivot = counts_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#rename headers
counts_pivot.columns = ['transcript_id', 'appellant/petitioner_count', 'appellee/respondent_count',
                       'scotus_justice_count']

#length of speaker speaking
length_pivot = pd.pivot_table(turns_combined[['transcript_id', 'text_duration', 'lawyer_side']],
                              index = 'transcript_id',
                              columns = 'lawyer_side',
                              values = 'text_duration',
                              aggfunc=np.sum,
                              fill_value=0)

#reset index
length_pivot = length_pivot.reset_index()

#drop columns with no lawyer side tags
length_pivot = length_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#rename headers
length_pivot.columns = ['transcript_id', 'appellant/petitioner_length', 'appellee/respondent_length',
                       'scotus_justice_length']

#concatenate pivots together
pivots_concate = pd.concat([speaker_role_text_pivot, 
                            counts_pivot[counts_pivot.columns[1:4]], 
                            length_pivot[length_pivot.columns[1:4]]],
                            axis=1,
                            join='inner')

#join verdict into df
train_test_df = pivots_concate.join(verdict.set_index('term_docket')['partyWinning'], on='transcript_id')

#remove NAs and blanks (these give errors when vectorizing)
train_test_df = train_test_df.dropna()

#create train and test split
x = train_test_df[['appellant/petitioner', 'appellee/respondent', 'scotus_justice',
                   'appellant/petitioner_count', 'appellee/respondent_count', 'scotus_justice_count',
                   'appellant/petitioner_length', 'appellee/respondent_length','scotus_justice_length']]
y = train_test_df.partyWinning
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

#count vectorizer
count_vect_1 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words='english', lowercase = False)
count_vect_2 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words='english', lowercase = False)
count_vect_3 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words='english', lowercase = False)

x_train_cv_petitioner_vec = count_vect_1.fit(x_train['appellant/petitioner'])
voc1 = count_vect_1.vocabulary_
x_train_cv_respondant_vec = count_vect_2.fit(x_train['appellee/respondent'])
voc2 = count_vect_2.vocabulary_
x_train_cv_scotus_justice_vec = count_vect_3.fit(x_train['scotus_justice'])

x_test_cv_petitioner = count_vect_1.transform(x_test['appellant/petitioner'])
x_test_cv_respondant = count_vect_2.transform(x_test['appellee/respondent'])
x_test_cv_scotus_justice = count_vect_3.transform(x_test['scotus_justice'])

x_train_cv_petitioner = pd.DataFrame(x_train_cv_petitioner_vec.transform(x_train['appellant/petitioner']).todense(),
                                        columns = x_train_cv_petitioner_vec.get_feature_names())
x_train_cv_respondant = pd.DataFrame(x_train_cv_respondant_vec.transform(x_train['appellee/respondent']).todense(),
                                        columns = x_train_cv_respondant_vec.get_feature_names())
x_train_cv_scotus_justice = pd.DataFrame(x_train_cv_scotus_justice_vec.transform(x_train['scotus_justice']).todense(),
                                        columns = x_train_cv_scotus_justice_vec.get_feature_names())
x_test_cv_petitioner = pd.DataFrame(x_test_cv_petitioner.todense(),
                                        columns = x_train_cv_petitioner_vec.get_feature_names())
x_test_cv_respondant = pd.DataFrame(x_test_cv_respondant.todense(),
                                        columns = x_train_cv_respondant_vec.get_feature_names())
x_test_cv_scotus_justice = pd.DataFrame(x_test_cv_scotus_justice.todense(),
                                        columns = x_train_cv_scotus_justice_vec.get_feature_names())

#put features into its own df for concatenate in next step
x_train_features = x_train[x_train.columns[3:9]]
x_train_features = x_train_features.reset_index()
x_test_features = x_test[x_train.columns[3:9]]
x_test_features = x_test_features.reset_index()

#change negative numbers into 0
x_train_features[x_train_features < 0 ] = 0
x_test_features[x_test_features < 0 ] = 0

#concatenate the bow back
x_train = pd.concat([x_train_cv_petitioner, 
                     x_train_cv_respondant, 
                     x_train_cv_scotus_justice,
                     x_train_features], 
                     axis = 1)
x_test = pd.concat([x_test_cv_petitioner, 
                    x_test_cv_respondant, 
                    x_test_cv_scotus_justice,
                    x_test_features],
                    axis = 1)

#reset index to match x
y_train = y_train.reset_index()
y_train = y_train.drop(['index'], axis=1)
x_train = x_train.drop(['index'], axis=1)
x_test = x_test.drop(['index'], axis=1)

#random forest
forest = RandomForestClassifier(n_estimators = 1000)
forest = forest.fit(x_train, y_train)
random_forest_prediction = forest.predict(x_test)
rf_accuracy = np.mean(random_forest_prediction == y_test)
print 'Accuracy of Bag-of-Words Random Forest Model:', round(rf_accuracy,4)

#Multinomial Naive Baynes
nb = MultinomialNB(alpha = 0.1).fit(x_train, y_train)
naive_baynes_prediction = nb.predict(x_test)
nb_accuracy = np.mean(naive_baynes_prediction == y_test)
print 'Accuracy of Bag-of-Words Multinomial Naive Baynes Model:', round(nb_accuracy,4)

# logistic regression
lr_model = LogisticRegression(C = 1, penalty = "l2")
lr_model_train = lr_model.fit(x_train, y_train)
lr_prediction = lr_model.predict(x_test)
lr_accuracy = np.mean(lr_prediction == y_test)

print 'Accuracy of Bag-of-Words Logistic Regression Model:', round(lr_accuracy,4)
print 'Actual Count of y_test Verdicts:\n', y_test.value_counts()
print 'RF predictions:', np.unique(random_forest_prediction, return_counts=True)
print 'NB predictions:', np.unique(naive_baynes_prediction, return_counts=True)
print 'LR predictions:', np.unique(lr_prediction, return_counts=True)

/home/pzhou11/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:169: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Accuracy of Bag-of-Words Random Forest Model: 0.6448
Accuracy of Bag-of-Words Multinomial Naive Baynes Model: 0.5672
Accuracy of Bag-of-Words Logistic Regression Model: 0.6404
Actual Count of y_test Verdicts:
1    575
0    340
Name: partyWinning, dtype: int64
RF predictions: (array(['0', '1'], dtype=object), array([ 91, 824]))
NB predictions: (array(['0', '1'], 
      dtype='|S1'), array([354, 561]))
LR predictions: (array(['0', '1'], dtype=object), array([137, 778]))


In [56]:
###tfidf vectorizer bag of words using scotus_justice, petitioner, respondent, and additional features
##features: chief justice indicator, petitioner code, respondent code, issue code, issue area code

#use original scdb winning party as verdict
#verdict value 0 = no favorable disposition for petitioning part apparent
#verdict value 1 = petitioning party received a favorable disposition
verdict = []
verdict_csv = csv.reader(open('../SCDB_2017_01_caseCentered_Citation.csv'))
for row in verdict_csv:
    docket_number = re.sub('-', '_', row[13])
    docket_number = re.sub(' ORIG', '_orig', docket_number)
    case_id = row[10]+'_'+docket_number
    verdict.append([case_id, row[12], row[17], row[19], row[36], row[39], row[40]])
    
verdict_header = verdict.pop(0)
verdict = pd.DataFrame(verdict, columns = verdict_header)
    
print "Verdict extraction done!"
    
#concate the turns files
turns_combined = pd.concat([turns1, turns2, turns3, turns4])

#remove _t01 and _t02 from transcript_id in turns_combined
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t01','')
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t02','')

#get advocate sides
advocates = pd.read_json('./advocate_dict.json')

advocate_turns = []

for index, row in turns_combined.iterrows():
    if row['speaker_role'] == 'scotus_justice':
        advocate_turns.append('scotus_justice')
    else:
        speaker = row['speaker']
        transcript_id = row['transcript_id']
        try:
            lawyer_side = advocates.ix[speaker][transcript_id]
            advocate_turns.append(lawyer_side)
        except:
            advocate_turns.append('None')

#insert advocate side to the turns_combined dataframe
turns_combined['lawyer_side'] = advocate_turns

print "Advocate side extraction done!"


################################################## GOOD PLACE TO SPLIT  ############################################

#pivot turns files by transcript_id and speaker_role for texts
speaker_role_text_pivot = turns_combined.pivot_table(index = 'transcript_id', 
                                                     columns = 'lawyer_side', 
                                                     values = 'text',
                                                     aggfunc=lambda x: ' '.join(x))
#reset index
speaker_role_text_pivot = speaker_role_text_pivot.reset_index()

#drop columns with no lawyer side tags
speaker_role_text_pivot = speaker_role_text_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#count of number of times speaker spoke
counts_pivot = pd.pivot_table(turns_combined[['transcript_id', 'lawyer_side']],
                              index = 'transcript_id',
                              columns = 'lawyer_side',
                              aggfunc=len,
                              fill_value=0)

#reset index
counts_pivot = counts_pivot.reset_index()

#drop columns with no lawyer side tags
counts_pivot = counts_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#rename headers
counts_pivot.columns = ['transcript_id', 'appellant/petitioner_count', 'appellee/respondent_count',
                       'scotus_justice_count']

#length of speaker speaking
length_pivot = pd.pivot_table(turns_combined[['transcript_id', 'text_duration', 'lawyer_side']],
                              index = 'transcript_id',
                              columns = 'lawyer_side',
                              values = 'text_duration',
                              aggfunc=np.sum,
                              fill_value=0)

#reset index
length_pivot = length_pivot.reset_index()

#drop columns with no lawyer side tags
length_pivot = length_pivot.drop(['NEED MORE INFO', 'None'], axis = 1)

#rename headers
length_pivot.columns = ['transcript_id', 'appellant/petitioner_length', 'appellee/respondent_length',
                       'scotus_justice_length']

#concatenate pivots together
pivots_concate = pd.concat([speaker_role_text_pivot, 
                            counts_pivot[counts_pivot.columns[1:4]], 
                            length_pivot[length_pivot.columns[1:4]]],
                            axis=1,
                            join='inner')

#Convert feature attributes to ints
verdict['petitioner'] = verdict['petitioner'].astype(int)
verdict['respondent'] = verdict.respondent.apply(lambda x: 0 if x == '' else x)
verdict['respondent'] = verdict['respondent'].astype(int)
verdict['issue'] = verdict.issue.apply(lambda x: 0 if x == '' else x)
verdict['issue'] = verdict['issue'].astype(int)
verdict['issueArea'] = verdict.issueArea.apply(lambda x: 0 if x == '' else x)
verdict['issueArea'] = verdict['issueArea'].astype(int)

#join verdict into df
train_test_df = pivots_concate.join(verdict.set_index('term_docket'), on='transcript_id')

#remove NAs and blanks (these give errors when vectorizing)
train_test_df = train_test_df.dropna()

#create train and test split
x = train_test_df[['appellant/petitioner', 'appellee/respondent', 'scotus_justice', 'chief',
                   'appellant/petitioner_count', 'appellee/respondent_count', 'scotus_justice_count',
                   'appellant/petitioner_length', 'appellee/respondent_length','scotus_justice_length',
                   'petitioner', 'respondent', 'issue', 'issueArea']]
y = train_test_df.partyWinning
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

print "Data re-shaping and combining done!"

#count vectorizer
stop_words = ["that", "the", "court", "of", "justice", "and", "please", "this", "to"]

count_vect_1 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words=stop_words, lowercase = False)
count_vect_2 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words=stop_words, lowercase = False)
count_vect_3 = TfidfVectorizer(ngram_range = (3,3), max_features = 2500, stop_words=stop_words, lowercase = False)
count_vect_chief = CountVectorizer()

print "Vectorizing done!"

x_train_cv_petitioner_vec = count_vect_1.fit(x_train['appellant/petitioner'])
voc1 = count_vect_1.vocabulary_
x_train_cv_respondant_vec = count_vect_2.fit(x_train['appellee/respondent'])
voc2 = count_vect_2.vocabulary_
x_train_cv_scotus_justice_vec = count_vect_3.fit(x_train['scotus_justice'])
x_train_chief = count_vect_chief.fit(x_train['chief'])

x_test_cv_petitioner = count_vect_1.transform(x_test['appellant/petitioner'])
x_test_cv_respondant = count_vect_2.transform(x_test['appellee/respondent'])
x_test_cv_scotus_justice = count_vect_3.transform(x_test['scotus_justice'])
x_test_chief = count_vect_chief.fit(x_test['chief'])

x_train_cv_petitioner = pd.DataFrame(x_train_cv_petitioner_vec.transform(x_train['appellant/petitioner']).todense(),
                                        columns = x_train_cv_petitioner_vec.get_feature_names())
x_train_cv_respondant = pd.DataFrame(x_train_cv_respondant_vec.transform(x_train['appellee/respondent']).todense(),
                                        columns = x_train_cv_respondant_vec.get_feature_names())
x_train_cv_scotus_justice = pd.DataFrame(x_train_cv_scotus_justice_vec.transform(x_train['scotus_justice']).todense(),
                                        columns = x_train_cv_scotus_justice_vec.get_feature_names())
x_train_cv_chief = pd.DataFrame(x_train_chief.transform(x_train['chief']).todense(),
                               columns = x_train_chief.get_feature_names())

x_test_cv_petitioner = pd.DataFrame(x_test_cv_petitioner.todense(),
                                        columns = x_train_cv_petitioner_vec.get_feature_names())
x_test_cv_respondant = pd.DataFrame(x_test_cv_respondant.todense(),
                                        columns = x_train_cv_respondant_vec.get_feature_names())
x_test_cv_scotus_justice = pd.DataFrame(x_test_cv_scotus_justice.todense(),
                                        columns = x_train_cv_scotus_justice_vec.get_feature_names())
x_test_cv_chief = pd.DataFrame(x_test_chief.transform(x_test['chief']).todense(),
                               columns = x_test_chief.get_feature_names())

#put features into its own df for concatenate in next step
x_train_features = x_train[x_train.columns[4:]]
x_train_features = x_train_features.reset_index()
x_test_features = x_test[x_train.columns[4:]]
x_test_features = x_test_features.reset_index()

#rest y train + test indices, drop 'index' columns, and convert to 1-d matrices
y_train = y_train.reset_index()
y_test = y_test.reset_index()
y_train = y_train.drop(['index'], axis = 1)
y_test = y_test.drop(['index'], axis = 1)
y_train = y_train.as_matrix()
y_test = y_test.as_matrix()
y_train = column_or_1d(y_train)
y_test = column_or_1d(y_test)

#change negative numbers into 0
x_train_features[x_train_features < 0 ] = 0
x_test_features[x_test_features < 0 ] = 0

#concatenate the bow back
x_train = pd.concat([x_train_cv_petitioner, 
                     x_train_cv_respondant, 
                     x_train_cv_scotus_justice,
                     x_train_cv_chief,
                     x_train_features], 
                     axis = 1)
x_test = pd.concat([x_test_cv_petitioner, 
                    x_test_cv_respondant, 
                    x_test_cv_scotus_justice,
                    x_test_cv_chief,
                    x_test_features],
                    axis = 1)

# #random forest
# forest = RandomForestClassifier(n_estimators = 1000, n_jobs = -1)
# forest = forest.fit(x_train, y_train)
# random_forest_prediction = forest.predict(x_test)
# rf_accuracy = np.mean(random_forest_prediction == y_test)
# print 'Accuracy of Bag-of-Words Random Forest Model:', round(rf_accuracy,4)

# #Multinomial Naive Baynes
# nb = MultinomialNB(alpha = 0.1).fit(x_train, y_train)
# naive_baynes_prediction = nb.predict(x_test)
# nb_accuracy = np.mean(naive_baynes_prediction == y_test)
# print 'Accuracy of Bag-of-Words Multinomial Naive Baynes Model:', round(nb_accuracy,4)

# logistic regression
lr_model = LogisticRegression(C = 0.1, penalty = "l1", n_jobs = -1)
lr_model_train = lr_model.fit(x_train, y_train)
lr_prediction = lr_model.predict(x_test)
lr_accuracy = np.mean(lr_prediction == y_test)

print 'Accuracy of Bag-of-Words Logistic Regression Model:', round(lr_accuracy,4)
print 'Actual Count of y_test Verdicts:\n', np.unique(y_test, return_counts=True)
# print 'RF predictions:', np.unique(random_forest_prediction, return_counts=True)
# print 'NB predictions:', np.unique(naive_baynes_prediction, return_counts=True)
print 'LR predictions:', np.unique(lr_prediction, return_counts=True)

Data re-shaping and combining done!
Vectorizing done!
Accuracy of Bag-of-Words Logistic Regression Model: 0.6612
Actual Count of y_test Verdicts:
(array(['0', '1'], dtype=object), array([323, 592]))
LR predictions: (array(['0', '1'], dtype=object), array([113, 802]))


In [57]:
print "Floor is:", round(575.0/(340.0+575.0), 4)

Floor is: 0.6284
